In [1]:
%run ../spark-default.py

In [2]:
spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic")

In [20]:
import builtins as b
from datetime import datetime, timedelta, date
import sys, traceback
from delta.tables import DeltaTable

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [11]:
ciha = spark.table("stage.ciha").filter("dt_atend is not null and diag_princ is not null and munic_res is not null")

ciha.printSchema()

# print(ciha.count())

ciha.limit(20).show()

root
 |-- mes_cmpt: string (nullable = true)
 |-- espec: string (nullable = true)
 |-- cgc_hosp: string (nullable = true)
 |-- munic_res: string (nullable = true)
 |-- nasc: date (nullable = true)
 |-- sexo: string (nullable = true)
 |-- uti_mes_to: string (nullable = true)
 |-- uti_int_to: string (nullable = true)
 |-- proc_rea: string (nullable = true)
 |-- qt_proc: string (nullable = true)
 |-- dt_atend: date (nullable = true)
 |-- dt_saida: date (nullable = true)
 |-- diag_princ: string (nullable = true)
 |-- diag_secun: string (nullable = true)
 |-- cobranca: string (nullable = true)
 |-- natureza: string (nullable = true)
 |-- gestao: string (nullable = true)
 |-- munic_mov: string (nullable = true)
 |-- cod_idade: string (nullable = true)
 |-- idade: string (nullable = true)
 |-- dias_perm: string (nullable = true)
 |-- morte: string (nullable = true)
 |-- nacional: string (nullable = true)
 |-- car_int: string (nullable = true)
 |-- homonimo: string (nullable = true)
 |-- cnes:

In [5]:
ciha.count()

138328339

In [12]:
municipios = spark.table("stage.municipios")

municipios.printSchema()

print(municipios.count())

municipios.show()

root
 |-- uf: string (nullable = true)
 |-- nome_uf: string (nullable = true)
 |-- regiao_geografica_intermediaria: string (nullable = true)
 |-- nome_regiao_geografica_intermediaria: string (nullable = true)
 |-- nome_regiao_geografica_imediata: string (nullable = true)
 |-- mesorregiao_geografica: string (nullable = true)
 |-- nome_mesorregiao: string (nullable = true)
 |-- microrregiao_geografica: string (nullable = true)
 |-- nome_microrregiao: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- cod_municipio_completo: string (nullable = true)
 |-- nome_municipio: string (nullable = true)
 |-- cod_municipio: string (nullable = true)

5570
+---+--------+-------------------------------+------------------------------------+-------------------------------+----------------------+-----------------+-----------------------+-----------------+---------+----------------------+--------------------+-------------+
| uf| nome_uf|regiao_geografica_intermediaria|nome_regiao_geogr

In [13]:
estados = spark.table("stage.estados")

estados.printSchema()

print(estados.count())

estados.show()

root
 |-- cod_uf: string (nullable = true)
 |-- nome_uf: string (nullable = true)
 |-- regiao: string (nullable = true)
 |-- sigla: string (nullable = true)

27
+------+-------------------+--------+-----+
|cod_uf|            nome_uf|  regiao|sigla|
+------+-------------------+--------+-----+
|    11|           Rondônia|   Norte|   RO|
|    12|               Acre|   Norte|   AC|
|    13|           Amazonas|   Norte|   AM|
|    14|            Roraima|   Norte|   RR|
|    15|               Pará|   Norte|   PA|
|    16|              Amapá|   Norte|   AP|
|    17|          Tocantins|   Norte|   TO|
|    21|           Maranhão|Nordeste|   MA|
|    22|              Piauí|Nordeste|   PI|
|    23|              Ceará|Nordeste|   CE|
|    24|Rio Grande do Norte|Nordeste|   RN|
|    25|            Paraíba|Nordeste|   PB|
|    26|         Pernambuco|Nordeste|   PE|
|    27|            Alagoas|Nordeste|   AL|
|    28|            Sergipe|Nordeste|   SE|
|    29|              Bahia|Nordeste|   BA|
|  

In [14]:
idhm = spark.table("stage.idhm")

idhm.printSchema()

print(idhm.count())

idhm.show()

root
 |-- cod_municipio: string (nullable = true)
 |-- idhm_2010: string (nullable = true)

5564
+-------------+---------+
|cod_municipio|idhm_2010|
+-------------+---------+
|       110001|    0.641|
|       110002|    0.702|
|       110003|     0.65|
|       110004|    0.718|
|       110005|    0.692|
|       110006|    0.685|
|       110007|    0.613|
|       110008|    0.611|
|       110009|    0.672|
|       110010|    0.657|
|       110011|    0.689|
|       110012|    0.714|
|       110013|    0.596|
|       110014|    0.643|
|       110015|    0.682|
|       110018|     0.71|
|       110020|    0.736|
|       110025|    0.664|
|       110026|    0.643|
|       110028|      0.7|
+-------------+---------+
only showing top 20 rows



In [15]:
ibge = spark.table("stage.ibge")

ibge.printSchema()

print(ibge.count())

ibge.show()

root
 |-- cod_municipio: string (nullable = true)
 |-- populacao_residente: string (nullable = true)
 |-- area_unidade_territorial: string (nullable = true)

5564
+-------------+-------------------+------------------------+
|cod_municipio|populacao_residente|area_unidade_territorial|
+-------------+-------------------+------------------------+
|       110001|              24392|                7067.025|
|       110002|              90353|                4426.571|
|       110003|               6313|                1314.362|
|       110004|              78574|                3792.801|
|       110005|              17029|                  2783.3|
|       110006|              18591|                1451.061|
|       110007|               8783|                3060.321|
|       110008|              13678|                4987.177|
|       110009|              28729|                4518.034|
|       110010|              41656|               24855.724|
|       110011|              52005|         

In [10]:
feriados = spark.table("stage.feriados")

feriados.printSchema()

print(feriados.count())

feriados.show()

root
 |-- data: string (nullable = true)
 |-- nome: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- descricao: string (nullable = true)
 |-- uf: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- cod_municipio: string (nullable = true)

154698
+----------+--------------------+---------+--------------------+---+----------------+-------------+
|      data|                nome|     tipo|           descricao| uf|       municipio|cod_municipio|
+----------+--------------------+---------+--------------------+---+----------------+-------------+
|2014-04-28|   Feriado Municipal|MUNICIPAL|                null| AC|     Acrelândia |       120001|
|2014-09-06|   Feriado Municipal|MUNICIPAL|                null| AC|     Acrelândia |       120001|
|2014-05-14|   Feriado Municipal|MUNICIPAL|                null| AC|   Assis Brasil |       120005|
|2014-05-31|   Feriado Municipal|MUNICIPAL|                null| AC|   Assis Brasil |       120005|
|2014-08-15|   Fer

In [11]:
feriados_prep = feriados \
                    .withColumn("facultativo", when(col("tipo") == lit("FACULTATIVO"), lit(True)).otherwise(lit(False))) \
                    .groupBy(
                        col("data")
                    ) \
                    .agg(
                        collect_list(
                            struct(
                                "data",
                                "nome",
                                "tipo",
                                "descricao",
                                "uf",
                                "municipio",
                                "cod_municipio"
                            )
                        ).alias("itens")
                    ) \
                    .rdd.map(lambda x: x.asDict()).collect()

feriados_prep = {
    x["data"]: [y.asDict() for y in x["itens"]]
    for x in feriados_prep
}

sorted_keys = list(feriados_prep.keys())
sorted_keys.sort()

feriados_prep = {
    k: feriados_prep[k]
    for k in sorted_keys
}

In [12]:
def checa_feriados(data, cod_municipio=None, uf_estado=None):
    feriados_possiveis = []
    erro = ""

    try:
        if data in feriados_prep.keys():
            for feriado in feriados_prep[data]:
                if feriado["tipo"] in ["NACIONAL", "FACULTATIVO"]:
                    feriados_possiveis.append(feriado)
                elif feriado["tipo"] in ["ESTADUAL"]:
                    if uf_estado is not None and feriado["uf"] == uf_estado:
                        feriados_possiveis.append(feriado)
                elif feriado["tipo"] in ["MUNICIPAL"]:
                    if cod_municipio is not None and feriado["cod_municipio"] == cod_municipio:
                        feriados_possiveis.append(feriado)
            
            feriado = True
            dist_feriado = 0
            
        if len(feriados_possiveis) == 0:
            feriado_info = None
            feriado = False
            
            mais_prox = ""
            dist_prox = 999
            for data_feriado in feriados_prep:
                data_obj = datetime.strptime(str(data), "%Y-%m-%d").date()
                feriado_obj = datetime.strptime(data_feriado, "%Y-%m-%d").date()
                d = b.abs((data_obj - feriado_obj).days)
                
                
                if d < dist_prox:
                    checked = False
                    for check_feriado in feriados_prep[data_feriado]:
                        if ((check_feriado["tipo"] in ["NACIONAL", "FACULTATIVO"])
                            or (check_feriado["tipo"] == "ESTADUAL" and check_feriado["uf"] == uf_estado)
                            or (check_feriado["tipo"] == "MUNICIPAL" and check_feriado["cod_municipio"] == cod_municipio)):
                            checked = True
                    if checked:
                        dist_prox = d
                        mais_prox = feriado
                        feriado_info = list(b.filter(None, [
                            x if ((x["tipo"] in ["NACIONAL", "FACULTATIVO"])
                                or (x["tipo"] == "ESTADUAL" and x["uf"] == uf_estado)
                                or (x["tipo"] == "MUNICIPAL" and x["cod_municipio"] == cod_municipio))
                            else None
                            for x in feriados_prep[data_feriado]
                        ]))
            
            dist_feriado = dist_prox 
            feriados_possiveis += feriado_info
            
    except Exception as e:
        ex_type, ex, tb = sys.exc_info()
        feriado = None
        dist_feriado = None
        feriados_possiveis = None
        erro = str(traceback.extract_tb(tb)) + " / " + str(e)

    return {
        "feriado": feriado,
        "distancia_feriado": dist_feriado,
        "feriado_info": feriados_possiveis,
        "erro": erro
    }

udf_checa_feriados = udf(checa_feriados, StructType([
    StructField("feriado", StringType(), True),
    StructField("distancia_feriado", IntegerType(), True),
    StructField("feriado_info", ArrayType(StructType([
        StructField("data", StringType(), True),
        StructField("nome", StringType(), True),
        StructField("tipo", StringType(), True),
        StructField("descricao", StringType(), True),
        StructField("uf", StringType(), True),
        StructField("municipio", StringType(), True),
        StructField("cod_municipio", StringType(), True)
    ])), True),
    StructField("erro", StringType(), True)
]))

In [13]:
prefinal = (ciha.alias("ciha")
            .join(municipios.alias("mun_1"), col("ciha.munic_res") == col("mun_1.cod_municipio"), "inner")
            .join(municipios.alias("mun_2"), col("ciha.munic_mov") == col("mun_2.cod_municipio"), "inner")
            .join(estados.alias("est_1"), col("mun_1.uf") == col("est_1.cod_uf"), "left")
            .join(estados.alias("est_2"), col("mun_2.uf") == col("est_2.cod_uf"), "left")
            # .join(idhm.alias("idhm"), col("ciha.munic_res") == col("idhm.cod_municipio"), "left")
            .join(ibge.alias("ibge_1"), col("ciha.munic_res") == col("ibge_1.cod_municipio"), "inner")
            .join(ibge.alias("ibge_2"), col("ciha.munic_mov") == col("ibge_2.cod_municipio"), "inner")

            .select(
                col("ciha.mes_cmpt"),
                col("ciha.espec"),
                col("ciha.cgc_hosp"),
                col("ciha.munic_res"),
                col("ciha.nasc"),
                col("ciha.sexo"),
                col("ciha.uti_mes_to"),
                col("ciha.uti_int_to"),
                col("ciha.proc_rea"),
                col("ciha.qt_proc"),
                col("ciha.dt_atend"),
                col("ciha.dt_saida"),
                col("ciha.diag_princ"),
                col("ciha.diag_secun"),
                col("ciha.cobranca"),
                col("ciha.natureza"),
                col("ciha.gestao"),
                col("ciha.munic_mov"),
                col("ciha.cod_idade"),
                col("ciha.idade"),
                col("ciha.dias_perm"),
                col("ciha.morte"),
                col("ciha.nacional"),
                col("ciha.car_int"),
                col("ciha.homonimo"),
                col("ciha.cnes"),
                col("ciha.fonte"),
                col("ciha.cgc_consor"),
                col("ciha.modalidade"),
                col("ciha.input_file_name"),
                col("ciha.estado_uf").alias("arquivo_sigla_uf"),
                col("ciha.ano_cmpt"),
                col("mun_1.nome_uf").alias("munic_res_nome_uf"),
                col("mun_2.nome_uf").alias("munic_mov_nome_uf"),
                col("mun_1.nome_municipio").alias("munic_res_nome_municipio"),
                col("mun_2.nome_municipio").alias("munic_mov_nome_municipio"),
                col("est_1.sigla").alias("munic_res_sigla_uf"),
                col("est_2.sigla").alias("munic_mov_sigla_uf"),
                col("est_1.regiao").alias("munic_res_regiao"),
                col("est_2.regiao").alias("munic_mov_regiao"),
                # col("idhm.idhm_2010").alias("idhm"),
                col("ibge_1.populacao_residente").alias("munic_res_populacao_residente"),
                col("ibge_2.populacao_residente").alias("munic_mov_populacao_residente"),
                col("ibge_1.area_unidade_territorial").alias("munic_res_area_unidade_territorial"),
                col("ibge_2.area_unidade_territorial").alias("munic_mov_area_unidade_territorial")
            )
           )

prefinal.limit(10).show()

+--------+-----+--------------+---------+----------+----+----------+----------+----------+-------+----------+----------+----------+----------+--------+--------+------+---------+---------+-----+---------+-----+--------+-------+--------+-------+-----+----------+----------+--------------------+----------------+--------+-----------------+-----------------+------------------------+------------------------+------------------+------------------+----------------+----------------+-----------------------------+-----------------------------+----------------------------------+----------------------------------+
|mes_cmpt|espec|      cgc_hosp|munic_res|      nasc|sexo|uti_mes_to|uti_int_to|  proc_rea|qt_proc|  dt_atend|  dt_saida|diag_princ|diag_secun|cobranca|natureza|gestao|munic_mov|cod_idade|idade|dias_perm|morte|nacional|car_int|homonimo|   cnes|fonte|cgc_consor|modalidade|     input_file_name|arquivo_sigla_uf|ano_cmpt|munic_res_nome_uf|munic_mov_nome_uf|munic_res_nome_municipio|munic_mov_nome

In [31]:
prefinal.count()

117092917

In [32]:
prefinal.filter("arquivo_sigla_uf != munic_mov_sigla_uf ").show(truncate=False)

+--------+-----+--------+---------+----+----+----------+----------+--------+-------+--------+--------+----------+----------+--------+--------+------+---------+---------+-----+---------+-----+--------+-------+--------+----+-----+----------+----------+---------------+----------------+--------+-----------------+-----------------+------------------------+------------------------+------------------+------------------+----------------+----------------+-----------------------------+-----------------------------+----------------------------------+----------------------------------+
|mes_cmpt|espec|cgc_hosp|munic_res|nasc|sexo|uti_mes_to|uti_int_to|proc_rea|qt_proc|dt_atend|dt_saida|diag_princ|diag_secun|cobranca|natureza|gestao|munic_mov|cod_idade|idade|dias_perm|morte|nacional|car_int|homonimo|cnes|fonte|cgc_consor|modalidade|input_file_name|arquivo_sigla_uf|ano_cmpt|munic_res_nome_uf|munic_mov_nome_uf|munic_res_nome_municipio|munic_mov_nome_municipio|munic_res_sigla_uf|munic_mov_sigla_uf|mun

In [38]:
preferiado = prefinal.select("dt_atend", "munic_res", "munic_mov", "arquivo_sigla_uf").distinct()
# preferiado.write.partitionBy("arquivo_sigla_uf").mode("overwrite").format("delta").option("mergeSchema", "true").save("s3a://datalake/temp/preferiado/")

In [40]:
preferiado.groupBy("arquivo_sigla_uf").count().show(999)

+----------------+-------+
|arquivo_sigla_uf|  count|
+----------------+-------+
|              SC| 167298|
|              RO|   8309|
|              PI|  16237|
|              AM|    205|
|              GO|  43843|
|              TO|   6087|
|              MT|  37731|
|              SP|1037767|
|              ES|  73907|
|              PB|  19211|
|              RS| 424524|
|              MS|  47872|
|              AL|  26075|
|              MG| 477283|
|              PA|  43608|
|              BA|  86589|
|              SE|  13603|
|              PE|  64279|
|              CE|  71862|
|              RN|  19454|
|              RJ| 135553|
|              MA|  15590|
|              AC|     63|
|              DF|   2201|
|              PR| 178625|
|              AP|   2981|
+----------------+-------+



In [15]:
ufs = [
    "SC",
    "RO",
    "PI",
    "AM",
    "GO",
    "TO",
    "MT",
    "SP",
    "ES",
    "PB",
    "RS",
    "MS",
    "AL",
    "MG",
    "PA",
    "BA",
    "SE",
    "PE",
    "CE",
    "RN",
    "RJ",
    "MA",
    "AC",
    "DF",
    "PR",
    "AP"
]

In [16]:
for uf in ufs:
    print(datetime.now().strftime("%Y-%m-%d %H:%M:%S") + ": " + uf)
    withferiado = spark.read.format("delta").load("s3a://datalake/temp/preferiado/").filter(f"arquivo_sigla_uf = '{uf}' ").withColumn("feriado", udf_checa_feriados(col("dt_atend"), col("munic_res"), col("arquivo_sigla_uf")))
    withferiado.write.partitionBy("arquivo_sigla_uf").mode("overwrite").format("delta").save("s3a://datalake/temp/withferiado1/")

2025-01-29 02:11:26: SP
2025-01-29 07:56:13: ES
2025-01-29 08:21:05: PB
2025-01-29 08:27:31: RS
2025-01-29 10:48:29: MS
2025-01-29 11:04:26: AL
2025-01-29 11:13:07: MG
2025-01-29 13:51:58: PA
2025-01-29 14:06:36: BA
2025-01-29 14:35:23: SE
2025-01-29 14:39:58: PE
2025-01-29 15:01:15: CE
2025-01-29 15:25:09: RN
2025-01-29 15:31:41: RJ
2025-01-29 16:16:45: MA
2025-01-29 16:21:56: AC
2025-01-29 16:22:00: DF
2025-01-29 16:22:49: PR
2025-01-29 17:23:01: AP


In [4]:
feriado_done = spark.read.format("delta").load("s3a://datalake/temp/withferiado1/")
feriado_done.filter("feriado.erro != '' ").show(20, truncate = False)

+----------+---------+---------+----------------+-----------------------------------------------------------------------------------------------------------------------------------------+
|dt_atend  |munic_res|munic_mov|arquivo_sigla_uf|feriado                                                                                                                                  |
+----------+---------+---------+----------------+-----------------------------------------------------------------------------------------------------------------------------------------+
|2001-01-26|353890   |353890   |SP              |{null, null, null, [<FrameSummary file /tmp/ipykernel_128/3567610216.py, line 51 in checa_feriados>] / 'NoneType' object is not iterable}|
|2000-09-03|352260   |352260   |SP              |{null, null, null, [<FrameSummary file /tmp/ipykernel_128/3567610216.py, line 51 in checa_feriados>] / 'NoneType' object is not iterable}|
|2004-07-13|355030   |355030   |SP              |{null, null

In [21]:
feriado_done.filter("feriado.erro != '' ").count()

296

In [5]:
feriado_done.count()

3020757

In [9]:
feriado_done.write.format("delta").save("s3a://datalake/stage/ciha_feriados/")

In [16]:
ciha_feriados = spark.read.format("delta").load("s3a://datalake/stage/ciha_feriados/")

In [18]:
ciha_step1 = (ciha.alias("ciha")
            .join(municipios.alias("mun_1"), col("ciha.munic_res") == col("mun_1.cod_municipio"), "inner")
            .join(municipios.alias("mun_2"), col("ciha.munic_mov") == col("mun_2.cod_municipio"), "inner")
            .join(estados.alias("est_1"), col("mun_1.uf") == col("est_1.cod_uf"), "left")
            .join(estados.alias("est_2"), col("mun_2.uf") == col("est_2.cod_uf"), "left")
            # .join(idhm.alias("idhm"), col("ciha.munic_res") == col("idhm.cod_municipio"), "left")
            .join(ibge.alias("ibge_1"), col("ciha.munic_res") == col("ibge_1.cod_municipio"), "inner")
            .join(ibge.alias("ibge_2"), col("ciha.munic_mov") == col("ibge_2.cod_municipio"), "inner")
            .join(ciha_feriados.alias("fer"), 
                  [col("ciha.dt_atend") == col("fer.dt_atend"), 
                  col("ciha.munic_res") == col("fer.munic_res"), 
                  col("ciha.munic_mov") == col("fer.munic_mov"), 
                  col("ciha.estado_uf") == col("fer.arquivo_sigla_uf")], 
                  "inner")

            .select(
                col("ciha.mes_cmpt"),
                col("ciha.espec"),
                col("ciha.cgc_hosp"),
                col("ciha.munic_res"),
                col("ciha.nasc"),
                col("ciha.sexo"),
                col("ciha.uti_mes_to"),
                col("ciha.uti_int_to"),
                col("ciha.proc_rea"),
                col("ciha.qt_proc"),
                col("ciha.dt_atend"),
                col("ciha.dt_saida"),
                col("ciha.diag_princ"),
                col("ciha.diag_secun"),
                col("ciha.cobranca"),
                col("ciha.natureza"),
                col("ciha.gestao"),
                col("ciha.munic_mov"),
                col("ciha.cod_idade"),
                col("ciha.idade"),
                col("ciha.dias_perm"),
                col("ciha.morte"),
                col("ciha.nacional"),
                col("ciha.car_int"),
                col("ciha.homonimo"),
                col("ciha.cnes"),
                col("ciha.fonte"),
                col("ciha.cgc_consor"),
                col("ciha.modalidade"),
                col("ciha.input_file_name"),
                col("ciha.estado_uf").alias("arquivo_sigla_uf"),
                col("ciha.ano_cmpt"),
                col("mun_1.nome_uf").alias("munic_res_nome_uf"),
                col("mun_2.nome_uf").alias("munic_mov_nome_uf"),
                col("mun_1.nome_municipio").alias("munic_res_nome_municipio"),
                col("mun_2.nome_municipio").alias("munic_mov_nome_municipio"),
                col("est_1.sigla").alias("munic_res_sigla_uf"),
                col("est_2.sigla").alias("munic_mov_sigla_uf"),
                col("est_1.regiao").alias("munic_res_regiao"),
                col("est_2.regiao").alias("munic_mov_regiao"),
                # col("idhm.idhm_2010").alias("idhm"),
                col("ibge_1.populacao_residente").alias("munic_res_populacao_residente"),
                col("ibge_2.populacao_residente").alias("munic_mov_populacao_residente"),
                col("ibge_1.area_unidade_territorial").alias("munic_res_area_unidade_territorial"),
                col("ibge_2.area_unidade_territorial").alias("munic_mov_area_unidade_territorial"),
                col("fer.feriado").alias("feriado")
            )
           )

ciha_step1.limit(10).show()

+--------+-----+--------------+---------+----------+----+----------+----------+----------+-------+----------+----------+----------+----------+--------+--------+------+---------+---------+-----+---------+-----+--------+-------+--------+-------+-----+----------+----------+--------------------+----------------+--------+-----------------+-----------------+------------------------+------------------------+------------------+------------------+----------------+----------------+-----------------------------+-----------------------------+----------------------------------+----------------------------------+--------------------+
|mes_cmpt|espec|      cgc_hosp|munic_res|      nasc|sexo|uti_mes_to|uti_int_to|  proc_rea|qt_proc|  dt_atend|  dt_saida|diag_princ|diag_secun|cobranca|natureza|gestao|munic_mov|cod_idade|idade|dias_perm|morte|nacional|car_int|homonimo|   cnes|fonte|cgc_consor|modalidade|     input_file_name|arquivo_sigla_uf|ano_cmpt|munic_res_nome_uf|munic_mov_nome_uf|munic_res_nome_mun

In [19]:
ciha_step1.write.partitionBy("ano_cmpt").format("delta").save("s3a://datalake/stage/ciha_step1/")

In [24]:
DeltaTable.createIfNotExists(spark) \
  .tableName("stage.ciha_step1") \
  .addColumns(ciha_step1.schema) \
  .partitionedBy("ano_cmpt") \
  .execute()

In [25]:
ciha_step1.write.insertInto("stage.ciha_step1")

In [26]:
spark.stop()

In [ ]:
# Até aqui, a base do CIHA foi enriquecida com:
#  - Nome das cidades (residencia e movimentação)
#  - Nome, uf e info dos estados (residencia e movimentação)
#  - Infos do IBGE como população e área
#  - Calculo de feriados